In [17]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [18]:
executable_path = {'executable_path': ChromeDriverManager().install()}

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/sjwyates/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [3]:
## NOTE: I used splinter for this first part because the HTML returned using requests.get was missing
## almost all the articles, including the first one. I assume this is because most of the articles
## are loaded via asynchronous JavaScript (looks like React), and requests is too impatient. 

url = 'https://mars.nasa.gov/news/'
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
articles = soup.find_all('div', class_='image_and_description_container')

In [6]:
article_1_title = articles[0].find('h3').get_text(strip=True)
article_1_text = articles[0].find('div', class_='article_teaser_body').get_text(strip=True)

In [7]:
print(article_1_title)
print('----------------------------------------------------------------')
print(article_1_text)

NASA Moves Forward With Campaign to Return Mars Samples to Earth
----------------------------------------------------------------
During this next phase, the program will mature critical technologies and make critical design decisions as well as assess industry partnerships.


In [8]:
base_url = 'https://www.jpl.nasa.gov'
url = base_url + '/spaceimages/?search=&category=Mars'
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

In [9]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [74]:
featured_image_url = base_url + soup.find('article', class_='carousel_item').attrs['style'].split("'")[1]

In [81]:
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA01384-1920x1200.jpg


In [2]:
import pandas as pd

In [49]:
mars_facts_df = pd.DataFrame(pd.read_html('https://space-facts.com/mars/')[0])

In [54]:
mars_facts_df.rename(columns={0: 'Description', 1: 'Mars'})

,Description,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [51]:
mars_facts_df.to_html(index=False, header=False, classes="table is-bordered is-striped is-hoverable is-full-width", justify="inherit")

'<table border="1" class="dataframe table is-bordered is-striped is-hoverable is-full-width">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [48]:
mars_facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    <

In [36]:
base_url = 'https://astrogeology.usgs.gov'
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(base_url + '/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

In [37]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [38]:
items = soup.find_all('div', class_='item')

In [39]:
links_to_results = []

for item in items:
    link = item.find('a', class_='itemLink product-item')['href']
    title = item.find('h3').get_text(strip=True)
    links_to_results.append({ 'link': link, 'title': title })

In [40]:
links_to_images = []

for link in links_to_results:
    browser.visit(base_url + link['link'])
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    downloads = soup.find('div', class_='downloads')
    img_url = downloads.find('ul').find_all('a')[1]['href']
    links_to_images.append({ 'title': link['title'], 'img_url': img_url })

In [41]:
print(links_to_images)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]
